In [28]:
import cv2
from matplotlib import pyplot as pt
import time
import numpy as np
from tensorflow import keras
import pickle

In [40]:
# cap = cv2.VideoCapture(0) #0 is video capture device
# ret, frame = cap.read()
# pt.imshow(frame)
labels = []
with open('labels.pkl', 'rb') as f:
    labels = pickle.load(f)
model = keras.models.load_model('test_model/model')
input_shape_model = tuple([int(x) for x in model.input.shape[1:]])
font = cv2.FONT_HERSHEY_SIMPLEX
  
# org
org = (00, 185)
  
# fontScale
fontScale = 1
   
# Red color in BGR
color = (0, 0, 255)
  
# Line thickness of 2 px
thickness = 2

In [54]:
#release the capture
#cap.release()
def predict(gray_image):
    X_test = cv2.resize(gray_image, (48, 48))
    #cv2.imshow('Processed',X_test)
    X_test = X_test/255.
    #plt.show(X_test)
    a3d = np.expand_dims(X_test, axis=2)
    X_test = np.array(a3d).reshape((-1,) + input_shape_model)
    #print(X_test.shape)
    pred = model.predict(X_test,verbose=0)
    classes = labels[pred.argmax()]
    return str(classes)

In [55]:
cap = cv2.VideoCapture(0)
prev_snap = 0
emotion="Neutral"
while cap.isOpened():
    snap_time = time.time()
    ret, frame = cap.read()
    if snap_time - prev_snap > 0.2:
        gray_image = cv2.cvtColor(frame ,cv2.COLOR_BGR2GRAY)
        emotion = predict(gray_image)
        prev_snap = time.time()
    image = cv2.putText(frame, emotion, org, font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow('Webcam',image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
end = time.time()
cv2.destroyAllWindows()
cap.release()